In [16]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

In [85]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo, Dream_report
from module1 import predict_for_month, find_best_threshold, predicted_labels, predicted_accuracy

In [18]:
len(Dream_report)

187652

##### MacroEcon
FX-Foriegn-exchange rate


CCI-台灣消費者信心指數

### 預計可使用的變數

In [19]:
# 名目、實質、時間
"""
券商預估、歷史營收增長
產業: 存貨量、產能利用率、訂單數量

MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--

sysallun
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\n券商預估、歷史營收增長\n產業: 存貨量、產能利用率、訂單數量\n\nMacroInfo\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n--\n實質國內生產毛額(GDP)–台幣 -> 只更新到202009\n實質GDP成長率(台灣) -> 只更新到202212\n--\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n\nsysallunTWESPF\n外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)\n景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)\n\nsysTaiStaValMAC\n景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)\n\n其他\n財報、營收\n'

### *【訓練步驟：資料處理 → 分割 train、test dataset → model fit  → 計算出機率&閾值 → 計算準確率】

### 總經資料處理

In [20]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_ICS.tail(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_GDPGrowth.head(3))

,ddate,value,category
916,202312,101.78,名目有效匯率指數–台灣
2069,202311,101.05,名目有效匯率指數–台灣
3315,202310,100.99,名目有效匯率指數–台灣


,ddate,value,category
258880,201403,80.96,台灣消費者信心指數
261195,201402,82.93,台灣消費者信心指數
263490,201401,80.83,台灣消費者信心指數


,ddate,value,category
316,202312,2.71,消費者物價指數(CPI)年增率
1357,202311,2.90,消費者物價指數(CPI)年增率
2575,202310,3.05,消費者物價指數(CPI)年增率


,ddate,value,category
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [139]:
MacroInfo.head()

,年月,代號,數值,名稱
0,202412,19600,1.06,經濟成長率(GDP)–單季
1,202412,19700,201644.00,國內生產毛額(GDP)–美元
2,202412,19710,6404219.00,國內生產毛額(GDP)–台幣
3,202412,19750,8567.00,平均每人國內生產毛額(GDP)–美元
4,202412,19760,272086.00,平均每人國內生產毛額(GDP)–台幣


,年月,代號,數值,名稱
1291,202311,12101,3.54,貨幣總額年增率(M1A)日平均
1299,202311,12501,5.33,貨幣總額年增率(M2)日平均
2506,202310,12101,3.76,貨幣總額年增率(M1A)日平均
2514,202310,12501,5.70,貨幣總額年增率(M2)日平均
3756,202309,12101,3.14,貨幣總額年增率(M1A)日平均
...,...,...,...,...
256574,201403,12501,5.89,貨幣總額年增率(M2)日平均
259082,201402,12101,9.43,貨幣總額年增率(M1A)日平均
259090,201402,12501,5.61,貨幣總額年增率(M2)日平均
261392,201401,12101,11.81,貨幣總額年增率(M1A)日平均


In [21]:
"""
# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) 
TradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]
TradingExceeds2 = TradingExceeds2.fillna(0)

# Create English Column names
TradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') 
#TradingExceeds2.head(10)
"""

"\n# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) \nTradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]\nTradingExceeds2 = TradingExceeds2.fillna(0)\n\n# Create English Column names\nTradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') \n#TradingExceeds2.head(10)\n"

### 三大法人買賣超資料處理

In [87]:
# 上市櫃公司三大法人買賣超、持股比率增減(分個股)
JuridicalInfo = JuridicalInfo.rename({'日期':'ddate', '股票代號':'stockid', '三大法人月初以來買賣超金額(千)':'GrandTransAmount(K)', '三大法人持股比率增減':'ShareRatio_change', '三大法人近1日買賣超金額(千)':'DailyTransAmount(K)'}, axis=1)
#display(JuridicalInfo.head())

In [100]:
JuridicalInfo['date'] = pd.to_datetime(JuridicalInfo['ddate']) # make sure the date column is datetime type
JuridicalInfo['last_date_of_month'] = JuridicalInfo.groupby([JuridicalInfo['date'].dt.year, JuridicalInfo['date'].dt.month])['date'].transform(max)  # get the last day of the month
# display(JuridicalInfo[JuridicalInfo['ddate']=='20230424'])


C:\Users\user1\AppData\Local\Temp\ipykernel_10524\796520960.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  JuridicalInfo['last_date_of_month'] = JuridicalInfo.groupby([JuridicalInfo['date'].dt.year, JuridicalInfo['date'].dt.month])['date'].transform(max)  # get the last day of the month


,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K),date,last_date_of_month
346052,20230424,1101,23899.0,0.05,-94045.0,2023-04-24,2023-04-28
346053,20230424,1101B,NaN,0.00,NaN,2023-04-24,2023-04-28
346054,20230424,1102,32471.0,0.01,30782.0,2023-04-24,2023-04-28
346055,20230424,1103,9927.0,0.05,6436.0,2023-04-24,2023-04-28
346056,20230424,1104,109863.0,0.27,46977.0,2023-04-24,2023-04-28
...,...,...,...,...,...,...,...
347865,20230424,9951,563.0,-0.03,-1253.0,2023-04-24,2023-04-28
347866,20230424,9955,3579.0,0.14,3539.0,2023-04-24,2023-04-28
347867,20230424,9958,1037642.0,-0.18,-76510.0,2023-04-24,2023-04-28
347868,20230424,9960,-15.0,0.00,0.0,2023-04-24,2023-04-28


In [24]:
TW_NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
TW_NominalGDP = TW_NominalGDP.rename(columns={'time':'ddate'})
#print(TW_NominalGDP.head(3))
TW_NominalGDP['ddate'] = TW_NominalGDP['ddate'].astype(str)

In [25]:
# 將GDP季資料先合併
GDP_quarterly = pd.merge(TW_NominalGDP, TW_GDPGrowth, on='ddate', how='left')
GDP_quarterly.columns = ['ddate', 'NominalGDP', 'GDP_growth', 'category_GDPgrowth']
#GDP_quarterly.head(3)

In [26]:
# (月)總經資料
dfs = [NominalExchageRate, TW_ICS, TW_CPIGrowth]
MergedMacroData = reduce(lambda left, right: pd.merge(left, right, on='ddate', how='left'), dfs)

# Rename columns
MergedMacroData.columns = ['ddate', 'FX', 'category_FX', 'CCI',
                          'category_CCI', 'CPI_growth', 'category_CPIgrowth']

#display(MergedMacroData)
# FX-Foriegn-exchange rate
# CCI-台灣消費者信心指數

In [27]:
MergedMacroData = pd.merge(MergedMacroData, GDP_quarterly, on='ddate', how='left')

In [28]:
MergedMacroData['NominalGDP'] = MergedMacroData['NominalGDP'].ffill()
MergedMacroData['GDP_growth'] = MergedMacroData['GDP_growth'].ffill()
MergedMacroData['category_GDPgrowth'] = MergedMacroData['category_GDPgrowth'].ffill()

In [29]:
MergedMacroData.head(9)

,ddate,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202312,101.78,名目有效匯率指數–台灣,70.06,台灣消費者信心指數,2.71,消費者物價指數(CPI)年增率,NaN,NaN,NaN
1,202311,101.05,名目有效匯率指數–台灣,69.40,台灣消費者信心指數,2.90,消費者物價指數(CPI)年增率,NaN,NaN,NaN
2,202310,100.99,名目有效匯率指數–台灣,69.06,台灣消費者信心指數,3.05,消費者物價指數(CPI)年增率,NaN,NaN,NaN
3,202309,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202308,100.67,名目有效匯率指數–台灣,67.51,台灣消費者信心指數,2.53,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
5,202307,101.31,名目有效匯率指數–台灣,68.39,台灣消費者信心指數,1.88,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
6,202306,102.80,名目有效匯率指數–台灣,66.66,台灣消費者信心指數,1.75,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
7,202305,101.84,名目有效匯率指數–台灣,63.67,台灣消費者信心指數,2.02,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
8,202304,101.35,名目有效匯率指數–台灣,63.18,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季


### 單純以總經變數訓練模型，準確率約51% -TheWhole

In [30]:
# Rename the column
df = df.rename(columns={'direction':'real_direction'})

In [31]:
# 合併df以及總經資料
TheWhole = pd.merge(df[['ddate', 'stockid', 'real_direction']].astype({'ddate':'str'}), 
                    MergedMacroData, on='ddate', how='left')
#TheWhole[TheWhole['ddate']=='202302'].head(3)

In [32]:
#TheWhole.columns
TheWhole = TheWhole.dropna()
#TheWhole.head()

In [33]:
# Split data by TimeSeriesSplit
tscv_macro = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_macro = list(tscv_macro.split(TheWhole))

In [34]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_macro = splits_macro[-1][0]
test_idx_macro = splits_macro[-1][1]

In [35]:
TargetCol = ['real_direction']
VarialbleCol = ['FX', 'CCI', 'CPI_growth',
                'NominalGDP', 'GDP_growth']

# Prepare data for separating
y_macro = TheWhole[TargetCol]
X_macro = TheWhole[VarialbleCol]

In [36]:
# Extract test and train data from overall data
X_train_macro, X_test_macro = X_macro.iloc[train_idx_macro], X_macro.iloc[test_idx_macro]
y_train_macro, y_test_macro = y_macro.iloc[train_idx_macro], y_macro.iloc[test_idx_macro]

In [37]:
# Train model based on the training data
model_macro = LogisticRegression()
model_fit_macro = model_macro.fit(X_train_macro, y_train_macro)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
# Make predictions on the test data
# Calculate thresholds to determine the predicted results
# probability
# display the results(0/1)

In [39]:
# 預測結果為1的機率
prob_macro = predict_for_month(model_fit_macro, TheWhole, VarialbleCol)
threshold_macro = find_best_threshold(TheWhole['real_direction'], prob_macro)
#print(round(threshold_macro, 2)) #0.51

print(round(threshold_macro, 4))

0.5102


In [40]:
# 以閾值為界判斷預測值應為0或1
pred_macro = predicted_labels(prob_macro, threshold_macro)

In [41]:
# 計算總經數值正確性
predicted_accuracy(TheWhole['real_direction'], pred_macro) #0.51

0.51

### 總經指數+營收 訓練+預測 -TheWhole_rev

In [42]:
TheWhole['ddate'] = TheWhole['ddate'].astype(str)
df['ddate'] = df['ddate'].astype(str)

In [43]:
df = df.rename({'direction':'real_direction'}, axis='columns')
TheWhole_rev = pd.merge(df, TheWhole, on=('ddate', 'stockid', 'real_direction'), how='inner')
TheWhole_rev.head(5)

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond12,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,-2.67,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,-1.06,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,12.55,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,-1.38,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,-11.13,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季


In [44]:
Varialble_MacroRev = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12', 'FX', 'CCI', 'CPI_growth',
                         'NominalGDP', 'GDP_growth']

Target_MacroRev = ['real_direction']

In [45]:
# Prepare data for separating
y_MacroRev = TheWhole_rev[Target_MacroRev]
X_MacroRev = TheWhole_rev[Varialble_MacroRev]

# Extract test and train data from overall data
X_train_MacroRev, X_test_MacroRev = X_MacroRev.iloc[train_idx_macro], X_MacroRev.iloc[test_idx_macro]
y_train_MacroRev, y_test_MacroRev = y_MacroRev.iloc[train_idx_macro], y_MacroRev.iloc[test_idx_macro]

In [46]:
# Train model based on the training data
model_MacroRev = LogisticRegression()
model_fit_MacroRev = model_MacroRev.fit(X_train_MacroRev, y_train_MacroRev)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
# 預測結果為1的機率
prob_MacroRev = predict_for_month(model_fit_MacroRev, TheWhole_rev, Varialble_MacroRev)
threshold_MacroRev = find_best_threshold(TheWhole_rev['real_direction'], prob_MacroRev) # 0.5092
print(threshold_MacroRev)
#print(round(threshold_MacroRev, 4))

# 以閾值為界判斷預測值應為0或1
pred_MacroRev = predicted_labels(prob_MacroRev, threshold_MacroRev)

0.5091613123764465


In [48]:
# 計算總經數值正確性
predicted_accuracy(TheWhole_rev['real_direction'], pred_MacroRev) # 0.62

0.62

In [49]:
# 分月份


### 以MLE判斷模型及變數是否有效

In [50]:
# 針對總經+12月營收判斷
# 使用 RFE 進行變數選擇
selector = RFE(model_MacroRev, n_features_to_select=20) #留下20 個 features
selector = selector.fit(X_MacroRev, y_MacroRev)

# Get the ranking of each feature
featre_ranking = selector.ranking_

featre_ranking
# 輸出選擇的特徵
selected_features = X_MacroRev.columns[selector.support_]
print(selected_features)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

Index(['revd1', 'yoyd1', 'mond1', 'yoyd2', 'yoyd3', 'mond3', 'yoyd4', 'mond4',
       'yoyd5', 'yoyd6', 'mond6', 'yoyd7', 'mond7', 'yoyd8', 'yoyd9', 'yoyd10',
       'mond11', 'revd12', 'yoyd12', 'mond12'],
      dtype='object')


In [51]:
selected_features = np.where(featre_ranking == 1)[0]

In [52]:
# 標準化
scaler = StandardScaler()
X_train_MacroRev = scaler.fit_transform(X_train_MacroRev)
X_test_MacroRev = scaler.transform(X_test_MacroRev)

model = LogisticRegression()
model.fit(X_train_MacroRev, y_train_MacroRev)

coeffients = model.coef_[0]
feature_importance = pd.DataFrame({'Feature': Varialble_MacroRev, 'Importance': np.abs(coeffients)})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
#feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))
display(feature_importance[:20])

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Feature,Importance
0,revd1,2.141860
33,revd12,1.481950
3,revd2,0.445347
15,revd6,0.383429
6,revd3,0.243646
18,revd7,0.217388
30,revd11,0.213559
12,revd5,0.156035
27,revd10,0.145844
2,mond1,0.075035


### 以重要度最高的20個變數訓練模型

In [53]:
#(feature_importance['Feature'][:20][0])
Varialble_import20 = list(feature_importance['Feature'][:20].astype(str))

In [54]:
Varialble_import20

['revd1',
 'revd12',
 'revd2',
 'revd6',
 'revd3',
 'revd7',
 'revd11',
 'revd5',
 'revd10',
 'mond1',
 'revd4',
 'CPI_growth',
 'CCI',
 'GDP_growth',
 'revd9',
 'yoyd1',
 'revd8',
 'NominalGDP',
 'FX',
 'mond7']

In [55]:
# Prepare data for separating
y_import20 = TheWhole_rev[Target_MacroRev]
X_import20 = TheWhole_rev[Varialble_import20]

# Separate data into test and train set
X_train_import20, X_test_import20 = X_import20.iloc[train_idx_macro], X_import20.iloc[test_idx_macro]
y_train_import20, y_test_import20 = y_import20.iloc[train_idx_macro], y_import20.iloc[test_idx_macro]

# Train model based on the training data
model_import20 = LogisticRegression()
model_fit_import20 = model_import20.fit(X_train_import20, y_train_import20)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
# 預測結果為1的機率
prob_import20 = predict_for_month(model_fit_import20, X_import20, Varialble_import20)
threshold_import20 = find_best_threshold(y_import20, prob_import20) # 0.5092
#print(threshold_import20)
#print(round(threshold_import20, 4)) 

# 以閾值為界判斷預測值應為0或1
pred_import20 = predicted_labels(prob_import20, threshold_import20)
#print(pred_import20)

# 計算總經數值正確性
predicted_accuracy(y_import20, pred_import20) # 0.62

0.62

###### Conclusion: because the threshold is almost the same as the unselect variable ones, the accuracy is the same.

### 以前20重要的變數+三大法人+其他總經變數

In [64]:
Dream_report['ddate'] = Dream_report['ddate'].astype(str)
Dream_report['stockid'] = Dream_report['stockid'].astype(int)

187652

In [70]:
Dream_report.columns

Index(['ddate', 'stockid', 'direction', 'ActualDirect'], dtype='object')

In [58]:
# 將TheWhole_rev結合夢幻報表的預測方向(direction)及實際變動方向(ActualDirect)
TheWhole_rev = TheWhole_rev.merge(Dream_report, how='inner', on=['ddate', 'stockid'])
TheWhole_rev

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth,direction,ActualDirect
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155167,201601,9951,0,340061.0,-4.64,19.32,356608.0,31.13,7.53,331642.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,1,0
155168,201601,9955,0,110743.0,-47.49,-63.31,210914.0,-16.54,86.76,112934.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0
155169,201601,9958,0,217788.0,-25.39,8.63,291900.0,19.28,-15.76,346505.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0
155170,201601,9960,0,33867.0,-20.50,-14.70,42599.0,-5.87,3.90,41000.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0


In [59]:
# Check for duplicate rows in TheWhole_rev
duplicates = TheWhole_rev.duplicated(subset=['ddate', 'stockid'], keep=False)
print("Number of duplicate rows in TheWhole_rev:", duplicates.sum())

Number of duplicate rows in TheWhole_rev: 0


In [60]:
if 'ActualDirect' in TheWhole_rev:
    print("Merge successful: 'ActualDirect' is in the merged dataframe.")
else:
    print("Merge unsuccessful: 'ActualDirect' is NOT in the merged dataframe.")

Merge successful: 'ActualDirect' is in the merged dataframe.


In [73]:
TheWhole_rev = TheWhole_rev.drop('real_direction', axis=1)

KeyError: "['real_direction'] not found in axis"

In [76]:
print(len(TheWhole_rev.columns))
print(len(TheWhole_rev))

52
155172


#### 將 Thewhole_rev 加上三大法人資料

In [89]:
# 將 Thewhole_rev 加上三大法人資料
len(JuridicalInfo) #6809784

6809784

In [95]:
JuridicalInfo.tail(3)
JuridicalInfo[JuridicalInfo['ddate'] == '20231222']

,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K),date,last_date_of_month
40638,20231222,1101,-1264831.0,-0.03,-10803.0,2023-12-22,2023-12-31
40639,20231222,1101B,NaN,0.00,NaN,2023-12-22,2023-12-31
40640,20231222,1102,-356754.0,-0.07,-130562.0,2023-12-22,2023-12-31
40641,20231222,1103,8766.0,0.00,350.0,2023-12-22,2023-12-31
40642,20231222,1104,160230.0,-0.01,-6544.0,2023-12-22,2023-12-31
...,...,...,...,...,...,...,...
42480,20231222,9951,-23316.0,-0.12,-5139.0,2023-12-22,2023-12-31
42481,20231222,9955,7873.0,-0.03,-848.0,2023-12-22,2023-12-31
42482,20231222,9958,245482.0,-0.18,-83704.0,2023-12-22,2023-12-31
42483,20231222,9960,-417.0,-0.01,-166.0,2023-12-22,2023-12-31


In [109]:
#JuridicalInfo_trans = JuridicalInfo
JuridicalInfo.columns
# 創建一個新的欄位 'year_month' 來儲存每個日期的年份和月份
JuridicalInfo['year_month'] = JuridicalInfo['date'].dt.to_period('M')

# 計算每個年份和月份的 'ShareRatio_change' 總和
JuridicalInfo['monthly_ShareRatio_change'] = JuridicalInfo.groupby(['year_month', 'stockid'])['ShareRatio_change'].transform('sum')

# 只在 'last_date_of_month' 等於 'date' 的那一行顯示 'monthly_ShareRatio_change'
JuridicalInfo.loc[JuridicalInfo['last_date_of_month'] != JuridicalInfo['date'], 'monthly_ShareRatio_change'] = None

JuridicalInfo

,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K),date,last_date_of_month,year_month,monthly_ShareRatio_change
0,20240124,1101,-1359231.0,-0.01,-70978.0,2024-01-24,2024-01-24,2024-01,-0.33
1,20240124,1101B,NaN,0.00,NaN,2024-01-24,2024-01-24,2024-01,0.00
2,20240124,1102,-556926.0,-0.01,-21277.0,2024-01-24,2024-01-24,2024-01,-0.36
3,20240124,1103,-10120.0,0.00,-397.0,2024-01-24,2024-01-24,2024-01,-0.16
4,20240124,1104,-22396.0,0.01,1907.0,2024-01-24,2024-01-24,2024-01,-0.13
...,...,...,...,...,...,...,...,...,...
6809779,20060102,9949,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6809780,20060102,9950,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6809781,20060102,9951,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6809782,20060102,9955,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN


In [120]:
# Create a boolean mask where 'date' is the same as 'last_date_of_month'
mask2 = (JuridicalInfo['date'] == JuridicalInfo['last_date_of_month'])

# Use the mask to index the DataFrame
JuridicalInfo_last_day = JuridicalInfo[mask2]
JuridicalInfo_last_day = JuridicalInfo_last_day.rename({'ddate': 'ddate_with_date'})
JuridicalInfo_last_day['ddate'] = JuridicalInfo_last_day['year_month'].dt.strftime('%Y%m')

#JuridicalInfo_last_day.columns
JuridicalInfo_last_day = JuridicalInfo_last_day.drop(columns=['ShareRatio_change','DailyTransAmount(K)', 'date','last_date_of_month','year_month'], axis=1)

,ddate,stockid,GrandTransAmount(K),monthly_ShareRatio_change
0,202401,1101,-1359231.0,-0.33
1,202401,1101B,NaN,0.00
2,202401,1102,-556926.0,-0.36
3,202401,1103,-10120.0,-0.16
4,202401,1104,-22396.0,-0.13
...,...,...,...,...
6789486,200601,9949,-137.0,-0.02
6789487,200601,9950,0.0,0.00
6789488,200601,9951,0.0,0.00
6789489,200601,9955,35232.0,1.49


In [125]:
JuridicalInfo_last_day.head()
#JuridicalInfo_last_day[JuridicalInfo_last_day['ddate'] == '202311']

,ddate,stockid,GrandTransAmount(K),monthly_ShareRatio_change
70109,202311,1101,-298562.0,-2.52
70110,202311,1101B,NaN,0.00
70111,202311,1102,927808.0,0.63
70112,202311,1103,44216.0,0.26
70113,202311,1104,86582.0,0.10
...,...,...,...,...
71942,202311,9951,-5535.0,-0.10
71943,202311,9955,-3803.0,-0.16
71944,202311,9958,-318319.0,-1.31
71945,202311,9960,2.0,0.00


In [137]:
type(JuridicalInfo_last_day['stockid'][0]) #str
type(TheWhole_rev['stockid'][0]) # int

JuridicalInfo_last_day['stockid'] = pd.to_numeric(JuridicalInfo_last_day['stockid'], errors='coerce')
JuridicalInfo_last_day = JuridicalInfo_last_day.dropna(subset=['stockid'])
JuridicalInfo_last_day['stockid'] = JuridicalInfo_last_day['stockid'].astype(int)
#JuridicalInfo_last_day.head()
#len(JuridicalInfo_last_day)

330890

In [151]:
TheWhole_rev_jur = pd.merge(TheWhole_rev, JuridicalInfo_last_day, how='inner', on=('ddate', 'stockid'))
TheWhole_rev_jur

,ddate,stockid,rev,mom,yoy,revd1,yoyd1,mond1,revd2,yoyd2,...,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth,direction,ActualDirect,GrandTransAmount(K),monthly_ShareRatio_change
0,202309,1101,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,-11.37,...,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0,-5255634.0,-1.75
1,202309,1102,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,-13.66,...,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1,98064.0,0.27
2,202309,1103,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,49.10,...,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0,-22378.0,-0.05
3,202309,1104,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,9.79,...,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1,-154952.0,-0.75
4,202309,1108,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,35.03,...,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0,-67511.0,-1.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151234,201601,9951,340061.0,-4.64,19.32,356608.0,31.13,7.53,331642.0,16.73,...,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,1,0,72370.0,0.55
151235,201601,9955,110743.0,-47.49,-63.31,210914.0,-16.54,86.76,112934.0,-60.74,...,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0,-773.0,-0.08
151236,201601,9958,217788.0,-25.39,8.63,291900.0,19.28,-15.76,346505.0,43.71,...,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0,-368.0,-0.02
151237,201601,9960,33867.0,-20.50,-14.70,42599.0,-5.87,3.90,41000.0,-15.58,...,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0,264.0,0.05


#### 以較重要的20個變數+三大法人訓練 & 計算準確性

In [138]:
# Jur_top20_var  = 
Varialble_import20

['revd1',
 'revd12',
 'revd2',
 'revd6',
 'revd3',
 'revd7',
 'revd11',
 'revd5',
 'revd10',
 'mond1',
 'revd4',
 'CPI_growth',
 'CCI',
 'GDP_growth',
 'revd9',
 'yoyd1',
 'revd8',
 'NominalGDP',
 'FX',
 'mond7']

In [148]:
Macro_currency_sales = MacroInfo[MacroInfo['名稱']. isin( ['貨幣總額年增率(M1A)日平均', '貨幣總額年增率(M2)日平均', 
                                                        '台灣外銷訂單金額–全球', '台灣外銷訂單金額–大陸及香港',
                                                        '台灣外銷訂單金額–資訊與通信產品', '台灣外銷訂單年增率',
                                                        '進口物價指數年增率'
                                                        ])]
Macro_prosperity = MacroInfo[MacroInfo['名稱']. isin( ['台灣景氣領先指標', '台灣景氣同時指標', '台灣景氣落後指標'])]

In [149]:
Macro_currency_sales = Macro_currency_sales.rename({'年月':'ddate',})

,年月,代號,數值,名稱
320,202312,18203,-2.89,進口物價指數年增率
1176,202312,TWA21,-16.02,台灣外銷訂單年增率
1193,202312,TWG15,12273.00,台灣外銷訂單金額–資訊與通信產品
1198,202312,TWL01,43814.00,台灣外銷訂單金額–全球
1203,202312,TWL12,10075.00,台灣外銷訂單金額–大陸及香港
...,...,...,...,...
261469,201401,18203,-0.41,進口物價指數年增率
263480,201401,TWA21,-2.77,台灣外銷訂單年增率
263515,201401,TWG15,9303.00,台灣外銷訂單金額–資訊與通信產品
263521,201401,TWL01,36109.00,台灣外銷訂單金額–全球
